In [1]:
import os
import re
import glob
import shutil
import numpy as np
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook

In [2]:
# get file list and paths
filepath = "/Users/ylin/My Drive/Cohost/Cohost Cleaner Compensation/Working/Data/"
property = pd.read_excel(filepath+"Property_Cohost.xlsx")
listings = property.Listing.unique().tolist()
Month ="2025-10"
oldloc = "/Users/ylin/My Drive/Cohost/Accounting/Company Transactions/2025/"+Month+"/"

def collect_files(oldloc):
    loc1 = os.listdir(oldloc)
    loc1.remove(".DS_Store")
    files = []
    #pathes = {'loc1':loc1,'fullpath':[oldloc+x for x in loc1]}
    for k in loc1:
        if os.path.isdir(oldloc+k) and not k.startswith('.DS_Store'):
            loc2 = os.listdir(oldloc+k)
            for d in loc2:
                files.append({'loc1': k, 'fullpath':oldloc+k, 'file': d})  
    return pd.DataFrame(files)
files = collect_files(oldloc)
#files

In [3]:
loc1 = os.listdir(oldloc)
loc1.remove(".DS_Store")
files = []
for k in loc1:
    if os.path.isdir(oldloc+k) and not k.startswith('.DS_Store'):
        loc2 = os.listdir(oldloc+k)
        for d in loc2:
            files.append({'loc1': k, 'fullpath':oldloc+k, 'file': d}) 
files= pd.DataFrame(files)
#test = files.fullpath+"/"+files.file
#print(f"It is : {test.iloc[0]}")
marks = [os.path.isdir(d) for d in files.fullpath+"/"+files.file]
subfolders = files.loc[marks,'fullpath']+"/"+files.loc[marks,'file']
subloc1 =files.loc[marks,'loc1']
#subfolders.iloc[0] # can see the entire name of the first subfolder
subfolders.tolist() # see the entire list of subfolders
subfiles =[]
for k,sub in zip(subloc1,subfolders):
    print(sub)
    loc2 = os.listdir(sub)
    for d in loc2:
        subfiles.append({'loc1': k, 'fullpath':sub, 'file': d})
subfiles= pd.DataFrame(subfiles)
files = pd.concat([files[~pd.Series(marks)], subfiles], ignore_index=True)

/Users/ylin/My Drive/Cohost/Accounting/Company Transactions/2025/2025-10/operations 7197/20251010 Marketing poulsbo bainbridge trip reimburse to jing 311.70


In [4]:
# Assign property names based on file naming conventions
files["property1"] = files["file"].str.split("_").str[1]
files["property"] = np.where(files["property1"].isin(listings), files["property1"],None)
files["property"] = np.where(files["file"].str.contains("Booking.com|booking.com", na=False),"BookingCommission",files["property"])
files["property"] = np.where(files["file"].str.contains('marketing|Marketing|Multi-units', na=False), "Valta Realty",files["property"])

txts = ["Longbranch","Hoodsport","Keaau","Lilliwaup","Beachwood","Seattle 906"]+["Beachwood " + str(i) for i in range(1, 11)]+\
         ["OSBR " + str(i) for i in range(1,11)]+["OSBR 12", "OSBR 11","OSBR",\
         "Microsoft C19","Microsoft 14645 C19","OSBR CS","OSBR C3","OSBR media room",\
         "Seatac 12834 Whole"]
#txts = [x for item in txts for x in (item if isinstance(item, list) else [item])]
chngs = ["Longbranch 6821","Hoodsport 26060","Keaau 15-1542","Lilliwaup 28610","Beachwood","Seattle 906 Lower"]+\
        ["Beachwood " + str(i) for i in range(1, 11)]+["Cottage " + str(i) for i in range(1, 11)]+\
        ["Cottage 12","Cottage 11 (tiny)","OSBR","Microsoft 14645-C19","Microsoft 14645-C19",\
          "OSBR","Cottage 3","OSBR","Seatac 12834"]
#chngs = [x for item in chngs for x in (item if isinstance(item, list) else [item])]

for k in range(len(txts)):
    idx = files["property1"] == txts[k]
    files.loc[idx, "property"] = chngs[k]
# for t, c in zip(txts, chngs):
#     idx = files['property1'] == t
#     files.loc[idx, 'property'] = c
    
txts = ["Ocean Spray Beach Resort","OSBR","Beachwood 1"]
chngs = ["OSBR","OSBR","Beachwood 1"]

for k in range(len(txts)):
    idx = files["file"].str.contains(txts[k], case=False, na=False) & files["property"].isna()
    files.loc[idx, "property"] = chngs[k]

mask1 = files["property1"].isin(["Maria", "Valta", "0SBR and Others Sirens cleaning"])
files.loc[mask1, "property"] = "Valta Realty"

# Fill remaining NaN property values with "Valta Realty"
files["property"] = files["property"].fillna("Valta Realty")

files.to_csv("/Users/ylin/My Drive/Cohost/Data and Reporting/04-Accounting/MonthlyInvoiceMigration/Filestracking/transactions_"+Month+".csv")

In [5]:
fileloc = pd.read_excel("/Users/ylin/My Drive/Cohost/"+"Data and Reporting/04-Accounting/"+\
                           "MonthlyInvoiceMigration/Data/FolderPaths.xlsx")
current_loc = "/Users/ylin/Google Drive/My Drive/Cohost/Accounting/* Monthly/"
fileloc.drop(columns=["property"], inplace=True)
files_loc = files.merge(fileloc, left_on="property", right_on="listing", how="left")
files_loc.loc[files_loc["loc"].isna(),'property']

Series([], Name: property, dtype: object)

In [6]:
# Copy files to their respective folders
for idx, row in files_loc.iterrows():
    src = row['fullpath']+"/"+row['file']
    if os.path.isdir(src):
        print(f"It is folder: {src}")
    elif pd.notna(row["loc"]) and row["property"] not in["BookingCommission","Valta Realty"]:   
        dst = current_loc+row['loc']+"/Invoice/"+row['file']
        shutil.copy2(src, dst)
    elif pd.notna(row['loc']) and row['property'] in ["Valta Realty", "BookingCommission"]:
        dst = row['loc']+"/"+row['file']
        shutil.copy2(src, dst)
  

In [7]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)

files_loc.iloc[[0]]
src = files_loc.iloc[0]['loc']+'/Invoice/'
#files_loc['loc'].unique()
#src=current_loc+'./WA Remote/Shelton 250 -- Angus Ni/2025'+"/Invoice/"
#os.listdir(src)


In [8]:
## Check copied files
newfiles=[]
for lc in files_loc['loc'].unique():
    mask = (files_loc["loc"] == lc) & (files_loc["property"] == "Valta Realty")
    if files_loc.loc[mask, "property"].empty:
        src = current_loc+lc+"/Invoice/"
        newfiles.extend(os.listdir(src))
    else:
        newfiles.extend(os.listdir(lc))


In [9]:
# Missing files after copying
print(files_loc.shape)
print(len(set(newfiles)))
set(files_loc['file']) - set(newfiles)


(250, 7)
1848


set()